```
data
    train
        class1
            img1.jpg
            img2.jpg
            ...
        class2
            img1.jpg
            img2.jpg
    test
        class1
            img1.jpg
            img2.jpg
            ...
        class2
            img1.jpg
            img2.jpg
```

In [15]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
transform = transforms.Compose([
    transforms.Resize((32,32)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))
])
train_dataset =  datasets.ImageFolder(root = './data/train', transform=transform)
test_dataset =  datasets.ImageFolder(root = './data/test', transform=transform)
train_loader = DataLoader(train_dataset,batch_size=5,shuffle=True)
test_loader = DataLoader(train_dataset,batch_size=5,shuffle=False)

In [23]:
img,label = next(iter(train_loader))
img.size(), label

(torch.Size([5, 3, 32, 32]), tensor([1, 0, 1, 1, 1]))

In [17]:
train_dataset.classes

['0', '1']

In [ ]:
import torch.nn as nn
from tqdm import tqdm
from torch.optim import Adam
class BasicBlock(nn.Module):
    def __init__(self, int_channels, out_channels, hidden_dim):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(int_channels, hidden_dim, kernel_size=3,padding=1)
        self.conv2 = nn.Conv2d(hidden_dim, out_channels,kernel_size=3,padding=1)
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool2d(2)
    def forward(self, x):
        x = self.relu( self.conv1(x) )
        x = self.relu( self.conv2(x) )
        out = self.pool(x)
        return out

import torch
class CNN(nn.Module):
    def __init__(self, num_class):
        super(CNN,self).__init__()
        self.block1 = BasicBlock(3,32,16)  
        self.block2 = BasicBlock(32,128,64)
        self.block3 = BasicBlock(128,256,128)

        # 분류기
        self.fc1 = nn.Linear( 4096, 2048)
        self.fc2 = nn.Linear(2048 , 256)
        self.fc3 = nn.Linear(256 , num_class)
        self.relu = nn.ReLU()
    def forward(self, x):
        x = self.block1(x)
        x = self.block2(x)
        x = self.block3(x)  #(N,256,4,4)        
        x = torch.flatten(x, start_dim=1)
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        out = self.fc3(x)
        return out    

device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = CNN(2)
model.to(device)

lr = 1e-3
optim = Adam(model.parameters(), lr=lr)
epochs = 20
# 학습루프
for epoch in range(epochs):
    for data, label in tqdm(train_loader,desc=f'epoch : {epoch+1}'):
        optim.zero_grad()
        preds = model(data.to(device))
        loss = nn.CrossEntropyLoss()(preds, label.to(device))
        loss.backward()
        optim.step()
    if (epoch+1) % 10 == 0:
        print(f'epoch : {epoch+1}  loss : {loss.item()}')

torch.save(model.state_dict(), 'hm.pth')    

epoch : 1:  53%|█████▎    | 109/206 [00:26<00:23,  4.08it/s]


KeyboardInterrupt: 